<b>Aktywność 6</b><br>
Niezbędne biblioteki: geopandas, pandas, pyproj

<b>Ćwiczenie 1.</b> Wczytaj dane z pliku ``shopping_centres.txt`` do obiektu typu DataFrame o nazwie <i>shopping_centres</i>.

In [24]:
import pandas as pd
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

shopping_centres = pd.read_csv('shopping_centres.txt', sep=';')

Zweryfikuj poprawność kodu.

In [25]:
assert isinstance(shopping_centres, pd.DataFrame)
for column in ('id', 'name', 'addr'):
    assert column in shopping_centres.columns

<b>Ćwiczenie 2.</b> Przeprowadź geokodowanie adresów z wykorzystaniem zmiennej <i>geocoded_addresses</i> i geokodera Nominatim.

In [26]:
from geopandas.tools import geocode

geocoded_addresses = geocode(shopping_centres['addr'], provider='nominatim', user_agent = 'my_app')

<b>Ćwiczenie 3.</b> Złącz wynik geokodowania z danymi wejściowymi i zapisz je w obiekcie typu GeoDataFrame o nazwie <i>shopping_centres</i>.

In [27]:
import geopandas as gpd

shopping_centres = shopping_centres.join(geocoded_addresses)

shopping_centres = gpd.GeoDataFrame(
    shopping_centres,
    geometry=shopping_centres.geometry
)


Zweryfikuj poprawność kodu.

In [28]:
assert isinstance(shopping_centres, gpd.GeoDataFrame)
for column in ('id', 'name', 'addr', 'geometry'):
    assert column in shopping_centres.columns

<b>Ćwiczenie 4.</b> Przeprowadź projekcję danych na EPSG:3879.

In [29]:
shopping_centres = shopping_centres.to_crs(epsg=3879)

Zweryfikuj poprawność kodu.

In [30]:
import pyproj

assert shopping_centres.crs == pyproj.CRS('EPSG:3879')

<b>Ćwiczenie 5.</b> Zapisz plik w formacie ShapeFile pod nazwą ``shopping_centres.shp``.

In [31]:
shopping_centres.to_file('shopping_centres.shp', driver='ESRI Shapefile')

<b>Ćwiczenie 6.</b> Oblicz bufor o promieniu 1.5 km od każdego centrum handlowego. Nadpisz wynik w kolumnie <i>geometry</i>.

In [32]:
shopping_centres['geometry'] = shopping_centres.geometry.buffer(1500)

Zweryfikuj poprawność kodu.

In [33]:
assert shopping_centres.geometry.geom_type.unique() == ['Polygon']

<b>Ćwiczenie 7.</b> Pobierz dane dotyczące populacji z wykorzystaniem zmiennej <i>population_grid</i>.

In [34]:
import geopandas as gpd
import requests
from io import BytesIO

url = 'https://kartta.hsy.fi/geoserver/wfs?service=wfs&version=2.0.0&request=GetFeature&typeName=asuminen_ja_maankaytto:Vaestotietoruudukko_2020&srsName=EPSG:3879'

response = requests.get(url, verify=False)

population_grid = gpd.read_file(BytesIO(response.content))

population_grid.crs = 'EPSG:3879'

print(population_grid.head())

C:\Users\rtome\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'kartta.hsy.fi'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


                       gml_id  index  asukkaita  asvaljyys  ika0_9  ika10_19  \
0  Vaestotietoruudukko_2020.1    703          5         51      99        99   
1  Vaestotietoruudukko_2020.2    710          8         44      99        99   
2  Vaestotietoruudukko_2020.3    711          5         90      99        99   
3  Vaestotietoruudukko_2020.4    715         13         34      99        99   
4  Vaestotietoruudukko_2020.5    848          5         53      99        99   

   ika20_29  ika30_39  ika40_49  ika50_59  ika60_69  ika70_79  ika_yli80  \
0        99        99        99        99        99        99         99   
1        99        99        99        99        99        99         99   
2        99        99        99        99        99        99         99   
3        99        99        99        99        99        99         99   
4        99        99        99        99        99        99         99   

                                            geometry  
0  POLY

Zweryfikuj poprawność kodu.

In [35]:
assert isinstance(population_grid, gpd.GeoDataFrame)
assert population_grid.crs == pyproj.CRS('EPSG:3879')

<b>Ćwiczenie 8.</b> Wykonaj odpowiednie złączenie przestrzenne pomiędzy obiektami <i>shopping_centres</i> i <i>population_grid</i> z wykorzystaniem zmiennej <i>populations</i>. Następnie usuń wszystkie kolumny poza <i>name</i> i <i>asukkaita</i> (<i>population</i>).

In [36]:
populations = gpd.sjoin(
    shopping_centres,
    population_grid,
    how='inner',
    predicate='intersects'
)
populations = populations[['name','asukkaita']]
populations.head()

,name,asukkaita
0,Itis,12
0,Itis,63
0,Itis,142
0,Itis,43
0,Itis,191


<b>Ćwiczenie 9.</b> Dla każdego centrum handlowego oblicz liczbę mieszkańców żyjących w promieniu 1.5 km od niego. Rezultat nadpisz w zmiennej <i>populations</i>.

In [37]:
populations = populations.groupby('name', as_index=False)['asukkaita'].sum()
populations.head(10)

    

,name,asukkaita
0,Forum,79188
1,Iso-omena,34441
2,Itis,30620
3,Jumbo,11449
4,REDI,42857
5,Sello,32226
6,Tripla,40388
